In [1]:
from pyquibbler import iquib, initialize_quibbler
initialize_quibbler()

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
from functools import partial
%matplotlib osx

In [2]:
# from colonycounter.image_processing_functions import search_for_blobs
# from colonycounter.plotting_functions import plot_circles

In [3]:
@partial(np.vectorize, signature='(),(m,n),()->()', otypes=[type(None)], is_graphics=True)
def show_image(ax, image, label=None):
    ax.imshow(image)
    if label is not None:
        ax.text(0.05, 0.95, label, color='w', ha='left', va='top', transform=ax.transAxes)

In [4]:
@partial(np.vectorize, otypes=[type(None)], is_graphics=True)
def plot_circles(ax, circle_list, args={"color": "white", "linewidth": 1, "alpha": 0.5} , is_graphics=True):
    for blob in circle_list:
        y, x, r = blob
        c = plt.Circle((x, y), r, **args, fill=False)
        ax.add_patch(c)

In [5]:
from skimage.feature import blob_log

@partial(np.vectorize, signature='(m,n),(),()->()', otypes=[object])
def find_colonies(image, params, label=None):
    if label is not None:
        print(f"processing image: {label}")
    blobs_log = blob_log(image, 
                         max_sigma=params['max_sigma'], 
                         min_sigma=params['min_sigma'],
                         num_sigma=params['num_sigma'],
                         overlap=params['overlap'],
                         threshold=params['threshold'], 
                         log_scale=True)
    blobs_log[:, 2] = blobs_log[:, 2] * np.sqrt(2)
    return blobs_log

In [6]:
# Prepare images
num_rows = 4
num_cols = 6
num_images = num_rows * num_cols

image = plt.imread('colonies1.png')
images = np.tile(image[:, :, 1], (num_images, 1, 1))

In [7]:
# Image labels
labels = np.arange(1, num_images)

In [8]:
# Colony-finding parameters
default_params = iquib({
    'min_sigma': 3,
    'max_sigma': 15,
    'num_sigma': 10,
    'overlap': 0.5,
    'threshold': 0.02})

default_params.allow_overriding = False

per_image_params = np.tile(default_params, num_images).setp(allow_overriding=True)

In [9]:
# Show parameters
def show_parameters(params, label=None):
    fig = plt.figure(figsize=(4, 2.5))
    gs = fig.add_gridspec(5, hspace=0.7, left=0.3)
    axs_sliders = gs.subplots()
    sliders_values = (
        ('min_sigma', 0, 30),
        ('max_sigma', 0, 30),
        ('num_sigma', 0, 30),
        ('overlap', 0, 1),
        ('threshold', 0, 1),
    )
    for num_slider, (name, valmin, valmax) in enumerate(sliders_values):
        Slider(ax=axs_sliders[num_slider], valmin=valmin, valmax=valmax, 
               valinit=params[name], label=name)
    if label is not None:
        axs_sliders[0].set_title(f'Parameters for image: {label}')
        

In [9]:
def show_parameters_by_index(index: int):
    show_parameters(per_image_params[index], labels[index])

In [18]:
# Prepare array of axes
fig = plt.figure(figsize=(8, 6))
gs = fig.add_gridspec(4, 6, hspace=0, wspace=0)
axs = gs.subplots()
axs = axs.ravel()
for ax in axs:
    ax.set_xticks([])
    ax.set_yticks([])

In [11]:
# Run find_colonies on all images
colonies = find_colonies(images, per_image_params, axs)

In [12]:
show_parameters_by_index(0)

In [13]:
show_image(axs[3], images[3], labels[3])

array(None, dtype=object)

In [17]:
plot_circles(axs[3:4], colonies[[3,4]])

np.vectorize(plot_circles)(array([<AxesSubplot:>], dtype=object), colonies[[3, 4]])

In [25]:
plt.figure()
ax = plt.gca()

In [26]:
show_image(ax, images[5], labels[5])

array(None, dtype=object)

In [27]:
plot_circles(ax, colonies[[5]])

np.vectorize(plot_circles)(<AxesSubplot:>, colonies[[5]])

In [28]:
show_parameters_by_index(5)